In [1]:
import openai

In [2]:
from openai import OpenAI

In [4]:
client = OpenAI()

In [5]:
import os

In [11]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [12]:
response.choices[0].message.content

"To provide a helpful answer, I would need more information about the specific course you're referring to, such as the enrollment deadlines, start date, and the institution or platform offering it. Generally, it's best to check the course webpage or contact the course provider directly to get the most accurate information regarding enrollment deadlines and any possibilities for late registration. If you're considering joining after the course has started, check if they allow latecomers and if there are any prerequisites you might need to catch up on."